In [1]:
import tensorflow as tf
import libspn as spn
import numpy as np

sess = tf.InteractiveSession()

In [2]:
v12 = spn.IVs(num_vars=2, num_vals=4)
v34 = spn.ContVars(num_vars=2)
v5 = spn.ContVars(num_vars=1)
s = spn.Sums((v12, [0, 5]), v34, (v12, [3]), v5, num_sums=2)
iv = s.generate_ivs()
w = s.generate_weights()
counts = tf.placeholder(tf.float32, shape=(None, 1))

op = s._compute_mpe_path(tf.identity(counts),
                         w.get_value(),
                         iv.get_value(),
                         v12.get_value(),
                         v34.get_value(),
                         v12.get_value(),
                         v5.get_value())


op_sum = s.get_value(spn.InferenceType.MARGINAL)

In [3]:
init = w.initialize()
counts_feed = [[10],
               [11],
               [12],
               [13],
               [14],
               [15],
               [16],
               [17]]

v12_feed = [[0, 1],
            [1, 1],
            [0, 0],
            [3, 3],
            [0, 1],
            [1, 1],
            [0, 0],
            [3, 3]]

v34_feed = [[0.1, 0.2],
            [1.2, 0.2],
            [0.1, 0.2],
            [0.9, 0.8],
            [0.1, 0.2],
            [1.2, 0.2],
            [0.1, 0.2],
            [0.9, 0.8]]

v5_feed = [[0.5],
           [0.5],
           [1.2],
           [0.9],
           [0.5],
           [0.5],
           [1.2],
           [0.9]]

#ivs_feed = [[-1], [-1], [-1], [-1], [1], [2], [3], [1]]
#ivs_feed = [[-1, -1], [-1, -1], [-1, -1], [-1, -1], [1, 1], [2, 2], [3, 3], [1, 1]]
ivs_feed = [[-1, 1], [-1, 1], [-1, 1], [-1, 1], [1, 1], [2, 1], [3, 1], [1, 1]]


In [4]:
with tf.Session() as sess:
    sess.run(init)
    out_sum = sess.run(op_sum, feed_dict={v12: v12_feed,
                                          v34: v34_feed,
                                          v5: v5_feed,
                                          iv: ivs_feed})
    
    print("Sum:", out_sum)


Sum: [[ 0.4666667   0.16666667]
 [ 0.48333335  0.16666667]
 [ 0.41666669  0.        ]
 [ 0.59999996  0.        ]
 [ 0.16666667  0.16666667]
 [ 0.20000002  0.16666667]
 [ 0.03333334  0.        ]
 [ 0.          0.        ]]


In [5]:
with tf.Session() as sess:
    sess.run(init)
    # Skip the IVs op
    out = sess.run(op, feed_dict={counts: counts_feed,
                                  iv: ivs_feed,
                                  v12: v12_feed,
                                  v34: v34_feed,
                                  v5: v5_feed})

In [6]:
# Weights
#np.testing.assert_array_almost_equal(
#    out[0], np.array([[10., 0., 0., 0., 0., 0.],
#                      [0., 0., 11., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 12.],
#                      [0., 0., 0., 0., 13., 0.],
#                      [0., 14., 0., 0., 0., 0.],
#                      [0., 0., 15., 0., 0., 0.],
#                      [0., 0., 0., 16., 0., 0.],
#                      [17., 0., 0., 0., 0., 0.]],
#                     dtype=np.float32))

print ("out[0] - Weights: \n", out[0])

out[0] - Weights: 
 [[[ 10.   0.   0.   0.   0.   0.]
  [  0.   0.  11.   0.   0.   0.]
  [  0.   0.   0.   0.   0.  12.]
  [  0.   0.   0.   0.  13.   0.]
  [  0.  14.   0.   0.   0.   0.]
  [  0.   0.  15.   0.   0.   0.]
  [  0.   0.   0.  16.   0.   0.]
  [ 17.   0.   0.   0.   0.   0.]]

 [[  0.  10.   0.   0.   0.   0.]
  [  0.  11.   0.   0.   0.   0.]
  [ 12.   0.   0.   0.   0.   0.]
  [ 13.   0.   0.   0.   0.   0.]
  [  0.  14.   0.   0.   0.   0.]
  [  0.  15.   0.   0.   0.   0.]
  [ 16.   0.   0.   0.   0.   0.]
  [ 17.   0.   0.   0.   0.   0.]]]


In [7]:
# IVs
#np.testing.assert_array_almost_equal(
#    out[1], np.array([[10., 0., 0., 0., 0., 0.],
#                      [0., 0., 11., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 12.],
#                      [0., 0., 0., 0., 13., 0.],
#                      [0., 14., 0., 0., 0., 0.],
#                      [0., 0., 15., 0., 0., 0.],
#                      [0., 0., 0., 16., 0., 0.],
#                      [17., 0., 0., 0., 0., 0.]],
#                     dtype=np.float32))

print ("out[1] - IVs: \n", out[1])

out[1] - IVs: 
 [[[ 10.   0.   0.   0.   0.   0.]
  [  0.   0.  11.   0.   0.   0.]
  [  0.   0.   0.   0.   0.  12.]
  [  0.   0.   0.   0.  13.   0.]
  [  0.  14.   0.   0.   0.   0.]
  [  0.   0.  15.   0.   0.   0.]
  [  0.   0.   0.  16.   0.   0.]
  [ 17.   0.   0.   0.   0.   0.]]

 [[  0.  10.   0.   0.   0.   0.]
  [  0.  11.   0.   0.   0.   0.]
  [ 12.   0.   0.   0.   0.   0.]
  [ 13.   0.   0.   0.   0.   0.]
  [  0.  14.   0.   0.   0.   0.]
  [  0.  15.   0.   0.   0.   0.]
  [ 16.   0.   0.   0.   0.   0.]
  [ 17.   0.   0.   0.   0.   0.]]]


In [8]:
#np.testing.assert_array_almost_equal(
#    out[2], np.array([[10., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 14., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [17., 0., 0., 0., 0., 0., 0., 0.]],
#                     dtype=np.float32))

print ("out[2]: \n", out[2])

out[2]: 
 [[ 10.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  14.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [ 17.   0.   0.   0.   0.   0.   0.   0.]]


In [9]:
#np.testing.assert_array_almost_equal(
#    out[3], np.array([[0., 0.],
#                      [11., 0.],
#                      [0., 0.],
#                      [0., 0.],
#                      [0., 0.],
#                      [15., 0.],
#                      [0., 16.],
#                      [0., 0.]],
#                     dtype=np.float32))

print ("out[3]: \n", out[3])

out[3]: 
 [[  0.   0.]
 [ 11.   0.]
 [  0.   0.]
 [  0.   0.]
 [  0.   0.]
 [ 15.   0.]
 [  0.  16.]
 [  0.   0.]]


In [10]:
#np.testing.assert_array_almost_equal(
#    out[4], np.array([[0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 13., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.]],
#                     dtype=np.float32))

print ("out[4]: \n", out[4])

out[4]: 
 [[  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.  13.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]]


In [ ]:
#np.testing.assert_array_almost_equal(
#    out[5], np.array([[0.],
#                      [0.],
#                      [12.],
#                      [0.],
#                      [0.],
#                      [0.],
#                      [0.],
#                      [0.]],
#                     dtype=np.float32))

print ("out[5]: \n", out[5])

In [ ]:
#np.testing.assert_array_almost_equal(
#    out[2], np.array([[10., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 14., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [17., 0., 0., 0., 0., 0., 0., 0.]],
#                     dtype=np.float32))

print ("out[6]: \n", out[6])

In [ ]:
#np.testing.assert_array_almost_equal(
#    out[3], np.array([[0., 0.],
#                      [11., 0.],
#                      [0., 0.],
#                      [0., 0.],
#                      [0., 0.],
#                      [15., 0.],
#                      [0., 16.],
#                      [0., 0.]],
#                     dtype=np.float32))

print ("out[7]: \n", out[7])

In [ ]:
#np.testing.assert_array_almost_equal(
#    out[4], np.array([[0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 13., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.],
#                      [0., 0., 0., 0., 0., 0., 0., 0.]],
#                     dtype=np.float32))

print ("out[8]: \n", out[8])

In [ ]:
#np.testing.assert_array_almost_equal(
#    out[5], np.array([[0.],
#                      [0.],
#                      [12.],
#                      [0.],
#                      [0.],
#                      [0.],
#                      [0.],
#                      [0.]],
#                     dtype=np.float32))

print ("out[9]: \n", out[9])